In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import xarray as xr

from datadrivencloud.utils import plot_xarray_chip
from datadrivencloud.data import ImageDataset, location_train_test_split
from datadrivencloud.modules import CloudModel
import pytorch_lightning as pl

from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp


## Test the dataset class

In [2]:
ds = xr.open_zarr('../data/train_zarr')
ds.lat.load()
ds.lon.load()
ds.datetime.load()
ds.location.load();

In [6]:
ds

,Array,Chunk
Bytes,114.73 GiB,1.50 MiB
Shape,"(11748, 5, 512, 512)","(1, 3, 256, 256)"
Count,187969 Tasks,93984 Chunks
Type,int64,numpy.ndarray


In [4]:
ds = ds.astype(int)#.isel(x=slice(0, 64), y=slice(0,64))

In [5]:
ds

,Array,Chunk
Bytes,114.73 GiB,1.50 MiB
Shape,"(11748, 5, 512, 512)","(1, 3, 256, 256)"
Count,187969 Tasks,93984 Chunks
Type,int64,numpy.ndarray


In [5]:
ds_train, ds_test = location_train_test_split(ds, 0.2, 12343124)

In [6]:
train_dataset = ImageDataset(ds_train)
test_dataset = ImageDataset(ds_test)

In [7]:
train_dataset[0]

(array([[[1526, 1534, 1542, ..., 7140, 6320, 6224],
         [1542, 1534, 1542, ..., 6776, 6320, 6276],
         [1544, 1534, 1540, ..., 6180, 6064, 5956],
         ...,
         [1614, 1664, 1698, ..., 1230, 1282, 1312],
         [1606, 1604, 1674, ..., 1248, 1290, 1316],
         [1626, 1604, 1668, ..., 1278, 1302, 1292]],
 
        [[1892, 1902, 1890, ..., 6844, 6176, 6156],
         [1898, 1902, 1896, ..., 6364, 6068, 6004],
         [1902, 1902, 1890, ..., 5836, 5700, 5692],
         ...,
         [1960, 2008, 2046, ..., 1458, 1526, 1574],
         [1952, 1982, 2024, ..., 1472, 1538, 1578],
         [1968, 1996, 2012, ..., 1494, 1526, 1526]],
 
        [[1890, 1890, 1884, ..., 6324, 5856, 5836],
         [1880, 1870, 1874, ..., 5812, 5664, 5636],
         [1854, 1842, 1826, ..., 5332, 5244, 5280],
         ...,
         [2094, 2124, 2158, ..., 1262, 1290, 1308],
         [2054, 2110, 2164, ..., 1268, 1286, 1294],
         [2072, 2122, 2138, ..., 1282, 1290, 1294]],
 
        [[101

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [9]:
device = "cuda:0"

In [21]:
model = smp.Unet(
    encoder_weights=None,
    in_channels=4,
    classes=1,
    activation='sigmoid'
).to(device)

In [22]:
learning_rate = 5e-6
for t in range(20):
    X, y = next(iter(train_dataloader))
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    y_pred = model(X.to(device)/2000.)

    # Compute and print loss
    loss = (y_pred - y.to(device)).pow(2).mean()
    if t % 4 == 0:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()


0 0.23288410902023315
4 0.2944332957267761
8 0.20847435295581818
12 0.29943183064460754
16 0.15163233876228333


In [20]:
y_pred[0]

tensor([[[-0.0389,  0.0662,  0.1975,  ...,  0.5582,  0.3214,  0.4406],
         [ 0.0153,  0.4491,  0.4022,  ...,  0.4133,  0.4029,  0.5014],
         [-0.1210,  0.3824,  0.3487,  ..., -0.1199,  0.1391,  0.3407],
         ...,
         [ 0.3372,  0.2202,  0.1341,  ...,  0.3001,  0.3768,  0.6512],
         [-0.0323,  0.4323,  0.3484,  ..., -0.2294,  0.0621,  0.1832],
         [-0.1512, -0.0886,  0.3297,  ...,  0.0013, -0.0029,  0.1379]]],
       device='cuda:0', grad_fn=<SelectBackward0>)